In [1]:
import scraper as s
import pandas as pd
import numpy as np
import datetime
import time
from scipy.stats import norm
m = s.stockMongo()

In [2]:
m.update_stockprices('AAPL')

TypeError: strptime() argument 1 must be str, not Timestamp

In [2]:
apple = m.get_options("AAPL")

In [3]:
apple.describe()

,ask,bid,last,strike,volume
count,178739.000000,178712.000000,177167.000000,178489.000000,160047.000000
mean,58.847093,57.847404,54.652596,283.967219,89.681544
std,87.013668,85.901368,82.011998,185.205138,170.146491
min,0.000000,0.000000,0.000000,18.750000,1.000000
25%,1.780000,1.600000,1.840000,120.000000,3.000000
50%,17.550000,17.000000,16.120000,250.000000,16.000000
75%,76.350000,75.000000,72.605000,420.000000,85.000000
max,523.000000,519.000000,963.000000,900.000000,999.000000


In [4]:
new_apple = apple.where(apple['last_trade']==apple.index.date).dropna()
#new_apple = new_apple[new_apple['volume'] > 50]

In [39]:
apl = m.get_stock_data('AAPL').drop_duplicates(subset ="date")

In [40]:
def split(apple):
    if apple['date'] < datetime.datetime.fromisoformat('2020-08-31'):
        return apple['c'] * 4
    else:
        return apple['c']

In [41]:
new_apple = apple.reset_index()
new_apple['time'] = new_apple['strike-date'] - new_apple['date']
new_apple['iv'] = pd.Series(new_apple['iv']).str.replace('%', '0', regex=False)
new_apple['iv'] = pd.to_numeric(new_apple['iv'],errors='coerce')
new_apple.describe()

,ask,bid,iv,last,strike,volume,time
count,178413.000000,178386.00000,176149.000000,176841.000000,178163.000000,159747.000000,178430
mean,58.954619,57.95312,76.453495,54.726049,284.273446,89.627411,175 days 21:38:41.637811
std,87.056728,85.94419,122.668041,82.064947,185.230039,170.112204,225 days 00:43:16.301023
min,0.000000,0.00000,0.000000,0.000000,18.750000,1.000000,-1 days +03:51:51.773000
25%,1.810000,1.63000,36.000000,1.840000,120.000000,3.000000,19 days 03:53:08.866000
50%,17.650000,17.15000,48.290000,16.150000,250.000000,16.000000,66 days 03:57:58.033000
75%,76.500000,75.10000,69.730000,72.800000,420.000000,85.000000,285 days 03:52:45.266000
max,523.000000,519.00000,999.880000,963.000000,900.000000,999.000000,800 days 03:37:08.107000


In [42]:
new_apple = new_apple.reset_index()
new_apple = new_apple.drop(['volume', 'in-money', 'bid', 'ask','last_trade', 'contract', 'index'], axis=1)
#new_apple = new_apple.set_index(['strike-date', 'type'])
new_apple['date'] = pd.to_datetime(new_apple['date']).dt.strftime('%m-%d-%Y')
new_apple['date'] = pd.to_datetime(new_apple['date'])
new_apple.head()

,date,iv,last,strike,strike-date,type,time
0,2020-08-02,148.44,284.24,95.0,2020-09-18,Call,46 days 00:20:22.974000
1,2020-08-02,0.00,254.30,100.0,2020-09-18,Call,46 days 00:20:22.974000
2,2020-08-02,152.54,264.25,105.0,2020-09-18,Call,46 days 00:20:22.974000
3,2020-08-02,129.69,266.25,110.0,2020-09-18,Call,46 days 00:20:22.974000
4,2020-08-02,0.00,255.30,115.0,2020-09-18,Call,46 days 00:20:22.974000


In [43]:
#new_apple = new_apple.reset_index()

new_apple['date'] = new_apple['date'].dt.normalize()
new_apple = pd.merge(new_apple, apl[['c', 'date']], on='date')
#apple2 = apple2.set_index('date')
new_apple['c'] = new_apple.apply(split, axis=1)
strike_pivot = new_apple.pivot_table(columns="strike-date", values="date", aggfunc=np.count_nonzero)
strike_cols = strike_pivot.columns
new_apple.head()

,date,iv,last,strike,strike-date,type,time,c
0,2020-08-03,219.48,272.27,170.0,2020-08-21,Call,17 days 03:57:58.005000,435.75
1,2020-08-03,210.65,206.99,180.0,2020-08-21,Call,17 days 03:57:58.005000,435.75
2,2020-08-03,0.00,175.69,190.0,2020-08-21,Call,17 days 03:57:58.005000,435.75
3,2020-08-03,193.31,194.12,195.0,2020-08-21,Call,17 days 03:57:58.005000,435.75
4,2020-08-03,189.79,167.15,200.0,2020-08-21,Call,17 days 03:57:58.005000,435.75


In [44]:
start = time.time()
new_apple5 = []
ivol = []

for k in range(len(strike_cols)):
    new_apple2 = new_apple.reset_index()
    new_apple2 = new_apple2.set_index(['strike-date'])
    print(strike_cols[k])
    new_apple3 = new_apple2.loc[strike_cols[k]]
    
    for day in range(max(new_apple3['time']).days):
    
        new_apple4 = new_apple3[new_apple3['time'].dt.days == day]

        if len(new_apple4) > 0:
            new_apple4 = new_apple4.reset_index()
            iv2 = abs(new_apple4['strike'] - new_apple4['c'])
            iv = new_apple4.loc[iv2.idxmin()]['iv']
            c = new_apple4.loc[iv2.idxmin()]['c']
            date = new_apple4.loc[iv2.idxmin()]['date']
            strike_date = new_apple4.loc[iv2.idxmin()]['strike-date']
            ivol.append([date, iv])

            new_apple2 = new_apple2.reset_index()
            new_apple2 = new_apple2.set_index(['type','strike-date'])
            apple_pivot_call = new_apple2.loc[('Call', strike_cols[k])]
            strike_pivot = apple_pivot_call.pivot_table(columns="strike", values="date", aggfunc=np.count_nonzero)
            strike_value_cols = strike_pivot.columns
            apple_pivot_put = new_apple2.loc[('Put', strike_cols[k])]
            strike_pivot = apple_pivot_put.pivot_table(columns="strike", values="date", aggfunc=np.count_nonzero)
            strike_value_puts = strike_pivot.columns

            cols = strike_value_cols[strike_value_cols > c]

            new_apple2 = new_apple2.reset_index()
            new_apple2 = new_apple2.set_index(['type', 'strike', 'strike-date'])

            for i in range(len(cols)-1):
                minput = abs(strike_value_puts - 2*c + cols[i])
                put = strike_value_puts[minput==min(minput)][0]
                #Calculation for second put (insurance)
                puts = strike_value_puts ==put
                puts2 = np.arange(0,len(puts), 1)
                put2 = strike_value_puts[puts2[puts]-1]

                aCLo = new_apple2.loc[('Call', cols[i], strike_cols[k])].drop_duplicates(subset ="date")[['last', 'date', 'c']]
                aCLo = aCLo.reset_index()
                idx = pd.date_range(min(aCLo['date']), max(aCLo['date']), freq = "D")
                aCLo.index = pd.DatetimeIndex(aCLo.date)
                aCLo = aCLo.reindex(idx,fill_value=None)
                aCLo = aCLo.interpolate(method ='linear', limit_direction ='forward')
                aCLo = aCLo.drop(['type', 'strike-date', 'date'], axis=1)
                aCLo = aCLo.reset_index()
                aCLo.rename(columns = {'last':'priceCLo', 'strike':'strikeCLo', 'index': 'date'}, inplace = True)

                aCHi = new_apple2.loc[('Call', cols[i+1], strike_cols[k])].drop_duplicates(subset ="date")[['last', 'date', 'c']]
                aCHi = aCHi.reset_index()
                idx = pd.date_range(min(aCHi['date']), max(aCHi['date']), freq = "D")
                aCHi.index = pd.DatetimeIndex(aCHi.date)
                aCHi = aCHi.reindex(idx,fill_value=None)
                aCHi = aCHi.interpolate(method ='linear', limit_direction ='forward')
                aCHi = aCHi.drop(['type', 'strike-date', 'date'], axis=1)
                aCHi = aCHi.reset_index()
                aCHi.rename(columns = {'last':'priceCHi', 'strike':'strikeCHi', 'index': 'date'}, inplace = True)

                aPLo = new_apple2.loc[('Put', put2, strike_cols[k])].drop_duplicates(subset ="date")[['last', 'date', 'c']]
                aPLo = aPLo.reset_index()
                idx = pd.date_range(min(aPLo['date']), max(aPLo['date']), freq = "D")
                aPLo.index = pd.DatetimeIndex(aPLo.date)
                aPLo = aPLo.reindex(idx,fill_value=None)
                aPLo = aPLo.interpolate(method ='linear', limit_direction ='forward')
                aPLo = aPLo.drop(['type', 'strike-date', 'date'], axis=1)
                aPLo = aPLo.reset_index()
                aPLo.rename(columns = {'last':'pricePLo', 'strike':'strikePLo', 'index': 'date'}, inplace = True)

                aPHi = new_apple2.loc[('Put', put, strike_cols[k])].drop_duplicates(subset ="date")[['last', 'date', 'c']]
                aPHi = aPHi.reset_index()
                idx = pd.date_range(min(aPHi['date']), max(aPHi['date']), freq = "D")
                aPHi.index = pd.DatetimeIndex(aPHi.date)
                aPHi = aPHi.reindex(idx,fill_value=None)
                aPHi = aPHi.interpolate(method ='linear', limit_direction ='forward')
                aPHi = aPHi.drop(['type', 'strike-date', 'date'], axis=1)
                aPHi = aPHi.reset_index()
                aPHi.rename(columns = {'last':'pricePHi', 'strike':'strikePHi', 'index': 'date'}, inplace = True)

                a = pd.merge(aCLo, aCHi, on=['date', 'c'])
                a = pd.merge(a, aPLo, on=['date', 'c'])
                a = pd.merge(a, aPHi, on=['date', 'c'])
                #a = a.set_index('date')
                a['condor'] = a['priceCLo'] - a['priceCHi'] + a['pricePHi'] - a['pricePLo']
                a['prob'] = (cols[i] - c) / c
                a['strike-date'] = strike_date
                a['condor-change'] = a['condor'].pct_change()
                a = a[a['date'] > date]
                a['start'] = day
                if len(a) >= 5:
                    new_apple5.append(a)

new_apple5 = pd.concat(new_apple5)
ivol = pd.DataFrame(data=ivol, columns=["date", "iv"])
end = time.time()
print(end - start)
    
    

2020-08-07 00:00:00


c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:26: PerformanceWarning: indexing past lexsort depth may impact performance.
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:29: PerformanceWarning: indexing past lexsort depth may impact performance.
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:46: PerformanceWarning: indexing past lexsort depth may impact performance.
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:56: PerformanceWarning: indexing past lexsort depth may impact performance.
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\pandas\core\indexing.py:1762: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:76: PerformanceWarning: indexing past lexsort depth may impact per

2020-08-14 00:00:00


c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:26: PerformanceWarning: indexing past lexsort depth may impact performance.
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:29: PerformanceWarning: indexing past lexsort depth may impact performance.
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:46: PerformanceWarning: indexing past lexsort depth may impact performance.
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:56: PerformanceWarning: indexing past lexsort depth may impact performance.
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\pandas\core\indexing.py:1762: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:76: PerformanceWarning: indexing past lexsort depth may impact per

2020-08-21 00:00:00


...

2022-06-17 00:00:00


c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:26: PerformanceWarning: indexing past lexsort depth may impact performance.
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:29: PerformanceWarning: indexing past lexsort depth may impact performance.
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:46: PerformanceWarning: indexing past lexsort depth may impact performance.
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:56: PerformanceWarning: indexing past lexsort depth may impact performance.
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\pandas\core\indexing.py:1762: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:76: PerformanceWarning: indexing past lexsort depth may impact per

2022-09-16 00:00:00


c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:26: PerformanceWarning: indexing past lexsort depth may impact performance.
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:29: PerformanceWarning: indexing past lexsort depth may impact performance.
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:46: PerformanceWarning: indexing past lexsort depth may impact performance.
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:56: PerformanceWarning: indexing past lexsort depth may impact performance.
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\pandas\core\indexing.py:1762: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:76: PerformanceWarning: indexing past lexsort depth may impact per

2023-01-20 00:00:00
3761.262153148651


In [56]:
new_apple5[new_apple5['start'] <= 60]['condor-change'].dropna().mean()

nan

In [29]:
strike_cols[16]

Timestamp('2020-11-27 00:00:00')

In [35]:
new_apple2.loc[strike_cols[19]]

,index,date,iv,last,strike,type,time,c
strike-date,,,,,,,,
2020-12-18,1969,2020-08-03,117.55,291.90,115.0,Call,136 days 03:57:22.648000,435.750000
2020-12-18,1970,2020-08-03,0.00,197.70,120.0,Call,136 days 03:57:22.648000,435.750000
2020-12-18,1971,2020-08-03,106.25,251.35,125.0,Call,136 days 03:57:22.648000,435.750000
2020-12-18,1972,2020-08-03,0.00,167.00,130.0,Call,136 days 03:57:22.648000,435.750000
2020-12-18,1973,2020-08-03,0.00,114.00,140.0,Call,136 days 03:57:22.648000,435.750000
...,...,...,...,...,...,...,...,...
2020-12-18,120614,2020-10-26,0.00,1.00,560.0,Put,52 days 03:51:24.560000,115.050003
2020-12-18,120615,2020-10-26,0.00,1.84,580.0,Put,52 days 03:51:24.560000,115.050003
2020-12-18,120616,2020-10-26,0.00,0.47,600.0,Put,52 days 03:51:24.560000,115.050003
